<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/LLM_GGUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- https://github.com/ggerganov/ggml/blob/master/docs/gguf.md gguf规范文档
- https://github.com/weedge/ggml/blob/master/docs/gguf-cn.md gguf规范中文版
- [用于量化 GGUF 模型的量化格式](https://github.com/ggerganov/ggml/blob/master/src/ggml-quants.h)。


# GGUF 工具


In [1]:
!git clone https://github.com/antirez/gguf-tools.git

Cloning into 'gguf-tools'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 136 (delta 81), reused 102 (delta 49), pack-reused 0
Receiving objects: 100% (136/136), 59.25 KiB | 4.94 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [74]:
%cd gguf-tools

/content/gguf-tools


In [3]:
!make

cc gguf-tools.c gguflib.c sds.c fp16.c \
	-march=native -flto -ffast-math \
	-g -ggdb -Wall -W -pedantic -O3 -o gguf-tools
gguf-tools.c: In function ‘gguf_tools_show’:
gguf-tools.c:169:36: warning: format ‘%llu’ expects argument of type ‘long long unsigned int’, but argument 5 has type ‘uint64_t’ {aka ‘long unsigned int’} []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wformat=]8;;]
  169 |         printf("%s tensor %.*s @%llu, %llu weights, %llu bytes\n",
      |                                 ~~~^
      |                                    |
      |                                    long long unsigned int
      |                                 %lu
......
  173 |             tensor.offset,
      |             ~~~~~~~~~~~~~           
      |                   |
      |                   uint64_t {aka long unsigned int}
gguf-tools.c:169:42: warning: format ‘%llu’ expects argument of type ‘long long unsigned int’, but argument 6 has type ‘uint64_t’ {ak

In [4]:
!pip3 install huggingface-hub

In [5]:
!huggingface-cli download TheBloke/phi-2-GGUF phi-2.Q8_0.gguf --local-dir ./models --local-dir-use-symlinks False


Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/phi-2-GGUF/resolve/main/phi-2.Q8_0.gguf to /root/.cache/huggingface/hub/tmp8bsvbj_7
phi-2.Q8_0.gguf: 100% 2.96G/2.96G [00:11<00:00, 250MB/s]
./models/phi-2.Q8_0.gguf


In [ ]:
!huggingface-cli download TheBloke/Llama-2-7b-Chat-GGUF --include='*Q4_K*gguf' --local-dir gguf-tools/models/ --local-dir-use-symlinks False

## gguf-tools show file.gguf
显示有关 GGUF 文件的详细信息。这将包括所有键值对，包括数组和详细的张量信息。张量的偏移量将相对于文件的开头（因此它们实际上是绝对偏移量），而不是像 GGUF 格式中的数据部分的开头。



In [75]:
!./gguf-tools show models/llama-2-7b-chat.Q4_K_M.gguf | head -100

models/llama-2-7b-chat.Q4_K_M.gguf (ver 2): 19 key-value pairs, 291 tensors
general.architecture: [string] llama
general.name: [string] LLaMA v2
llama.context_length: [uint32] 4096
llama.embedding_length: [uint32] 4096
llama.block_count: [uint32] 32
llama.feed_forward_length: [uint32] 11008
llama.rope.dimension_count: [uint32] 128
llama.attention.head_count: [uint32] 32
llama.attention.head_count_kv: [uint32] 32
llama.attention.layer_norm_rms_epsilon: [float32] 0.000001
general.file_type: [uint32] 15
tokenizer.ggml.model: [string] llama
tokenizer.ggml.tokens: [array] [<unk>, <s>, </s>, <0x00>, <0x01>, <0x02>, <0x03>, <0x04>, <0x05>, <0x06>, <0x07>, <0x08>, <0x09>, <0x0A>, <0x0B>, <0x0C>, <0x0D>, <0x0E>, <0x0F>, <0x10>, <0x11>, <0x12>, <0x13>, <0x14>, <0x15>, <0x16>, <0x17>, <0x18>, <0x19>, <0x1A>, ... 31970 more items of 32000]
tokenizer.ggml.scores: [array] [0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000

In [9]:
!./gguf-tools show models/phi-2.Q8_0.gguf | head -100

models/phi-2.Q8_0.gguf (ver 3): 20 key-value pairs, 325 tensors
general.architecture: [string] phi2
general.name: [string] Phi2
phi2.context_length: [uint32] 2048
phi2.embedding_length: [uint32] 2560
phi2.feed_forward_length: [uint32] 10240
phi2.block_count: [uint32] 32
phi2.attention.head_count: [uint32] 32
phi2.attention.head_count_kv: [uint32] 32
phi2.attention.layer_norm_epsilon: [float32] 0.000010
phi2.rope.dimension_count: [uint32] 32
general.file_type: [uint32] 7
tokenizer.ggml.add_bos_token: [bool] false
tokenizer.ggml.model: [string] gpt2
tokenizer.ggml.tokens: [array] [!, ", #, $, %, &, ', (, ), *, +, ,, -, ., /, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, :, ;, <, =, >, ... 51170 more items of 51200]
tokenizer.ggml.token_type: [array] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 51170 more items of 51200]
tokenizer.ggml.merges: [array] [Ġ t, Ġ a, h e, i n, r e, o n, Ġt he, e r, Ġ s, a t, Ġ w, Ġ o, e n, Ġ c, i t, i s, a n, o r, e s, Ġ b, e 

## gguf-tools compare file1.gguf file2.gguf
此工具有助于了解两个 LLM（或其他以 GGUF 文件形式分发的模型）是否相关，例如一个是否是另一个的微调，或者两者是否都是从同一个父模型进行微调的。

对于每个匹配的张量（名称和参数数量相同），该命令计算平均权重差异（以百分比表示，因此在 -N 到 +N 范围内的随机分布平均上与另一个随机分布相比较将达到 100% 差异）。这有助于查看模型是否是另一个模型的微调版本，它被微调了多少，哪些层在微调时被冻结等。请注意，由于量化，即使功能上等效的张量可能也存在一些小的平均差异。

示例输出：


In [10]:
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q8_0.gguf \
  --local-dir ./models --local-dir-use-symlinks False


Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q8_0.gguf to /root/.cache/huggingface/hub/tmpexfcef09
mistral-7b-instruct-v0.2.Q8_0.gguf: 100% 7.70G/7.70G [00:28<00:00, 273MB/s]
./models/mistral-7b-instruct-v0.2.Q8_0.gguf


In [11]:
!huggingface-cli download TheBloke/SOLAR-10.7B-Instruct-v1.0-uncensored-GGUF solar-10.7b-instruct-v1.0-uncensored.Q8_0.gguf \
  --local-dir ./models --local-dir-use-symlinks False


Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/SOLAR-10.7B-Instruct-v1.0-uncensored-GGUF/resolve/main/solar-10.7b-instruct-v1.0-uncensored.Q8_0.gguf to /root/.cache/huggingface/hub/tmp6ymjwy_8
(…)10.7b-instruct-v1.0-uncensored.Q8_0.gguf: 100% 11.4G/11.4G [03:57<00:00, 48.0MB/s]
./models/solar-10.7b-instruct-v1.0-uncensored.Q8_0.gguf


In [12]:
!./gguf-tools compare models/mistral-7b-instruct-v0.2.Q8_0.gguf models/solar-10.7b-instruct-v1.0-uncensored.Q8_0.gguf


[token_embd.weight]: avg weights difference: 44.539944%
[blk.0.attn_q.weight]: avg weights difference: 48.717736%
[blk.0.attn_k.weight]: avg weights difference: 56.201885%
[blk.0.attn_v.weight]: avg weights difference: 47.087249%
[blk.0.attn_output.weight]: avg weights difference: 47.663048%
[blk.0.ffn_gate.weight]: avg weights difference: 37.508761%
[blk.0.ffn_up.weight]: avg weights difference: 39.061584%
[blk.0.ffn_down.weight]: avg weights difference: 39.632648%
[blk.0.attn_norm.weight]: avg weights difference: 3.997229%
[blk.0.ffn_norm.weight]: avg weights difference: 5.686371%
[blk.1.attn_q.weight]: avg weights difference: 47.318189%
[blk.1.attn_k.weight]: avg weights difference: 44.031983%
[blk.1.attn_v.weight]: avg weights difference: 36.303868%
[blk.1.attn_output.weight]: avg weights difference: 40.389659%
[blk.1.ffn_gate.weight]: avg weights difference: 38.658373%
[blk.1.ffn_up.weight]: avg weights difference: 38.838259%
[blk.1.ffn_down.weight]: avg weights difference: 37.840

In [76]:
!./gguf-tools compare models/llama-2-7b-chat.Q4_K_M.gguf models/llama-2-7b-chat.Q4_K_S.gguf


[token_embd.weight]: avg weights difference: 0.000000%
[blk.0.attn_norm.weight]: avg weights difference: 0.000000%
[blk.0.ffn_down.weight]: dequantization function missing...
[blk.0.ffn_gate.weight]: avg weights difference: 0.000000%
[blk.0.ffn_up.weight]: avg weights difference: 0.000000%
[blk.0.ffn_norm.weight]: avg weights difference: 0.000000%
[blk.0.attn_k.weight]: avg weights difference: 0.000000%
[blk.0.attn_output.weight]: avg weights difference: 0.000000%
[blk.0.attn_q.weight]: avg weights difference: 0.000000%
[blk.0.attn_v.weight]: dequantization function missing...
[blk.1.attn_norm.weight]: avg weights difference: 0.000000%
[blk.1.ffn_down.weight]: dequantization function missing...
[blk.1.ffn_gate.weight]: avg weights difference: 0.000000%
[blk.1.ffn_up.weight]: avg weights difference: 0.000000%
[blk.1.ffn_norm.weight]: avg weights difference: 0.000000%
[blk.1.attn_k.weight]: avg weights difference: 0.000000%
[blk.1.attn_output.weight]: avg weights difference: 0.000000%
[b

## gguf-tools inspect-tensor file.gguf tensor.name [count]
显示指定张量的所有权重值（如果未指定 count，则仅显示前 count 个）。这对于低级别的任务很有用，比如检查量化是否按预期工作，查看引入的误差，模型指纹等。



In [20]:
!echo -e "\n----token_embd.weight----\n"
!./gguf-tools inspect-tensor models/mistral-7b-instruct-v0.2.Q8_0.gguf token_embd.weight 50
!echo -e "\n----blk.0.attn_q.weight----\n"
!./gguf-tools inspect-tensor models/mistral-7b-instruct-v0.2.Q8_0.gguf blk.0.attn_q.weight 50


----token_embd.weight----

0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 0.000000, 0.000000, 
0.000000, 0.000000, 

----blk.0.attn_q.weight----

0.000085, 0.000766, 0.000000, -0.000680, 
0.001616, -0.000085, 0.003828, 0.000255, 
0.002637, 0.000000, 0.000425, -0.000085, 
0.000000, 0.000170, 0.006039, -0.000595, 
0.000000, -0.002892, 0.000000, -0.006294, 
-0.004848, -0.002382, 0.010802, 0.002892, 
0.001191, 0.000000, -0.000255, 0.000000, 
0.000255, -0.003402, 0.000000, 0.007400, 
0.000000, 0.000000, 0.002815, 0.000088, 
0.000000, 0.000880, 0.000000, 0.000000, 
0.000000,

## gguf-tools split-mixtral 65230776370407150546470161412165 mixtral.gguf out.gguf
从 Mixtral 7B MoE 中提取一个 7B 模型 out.gguf，使用指定的每层 MoE ID（在序列 652... 中有 32 个数字）。

请注意，通过 split-mixtral 方式获得的模型实际上没有执行任何有用的工作。这只是一个实验和一个不太重要的任务，用来展示如何使用该库。很可能很快就会被移除，一旦我有更有趣和有用的示例要展示，比如模型合并。

In [21]:
!./gguf-tools split-mixtral 65230776370407150546470161412165 \
  models/mistral-7b-instruct-v0.2.Q8_0.gguf models/mistral-7b-instruct-v0.2.Q8_0_out.gguf

Copying general.architecture
Copying general.name
Copying llama.context_length
Copying llama.embedding_length
Copying llama.block_count
Copying llama.feed_forward_length
Copying llama.rope.dimension_count
Copying llama.attention.head_count
Copying llama.attention.head_count_kv
Copying llama.attention.layer_norm_rms_epsilon
Copying llama.rope.freq_base
Copying general.file_type
Copying tokenizer.ggml.model
Copying tokenizer.ggml.tokens
Copying tokenizer.ggml.scores
Copying tokenizer.ggml.token_type
Copying tokenizer.ggml.bos_token_id
Copying tokenizer.ggml.eos_token_id
Copying tokenizer.ggml.unknown_token_id
Copying tokenizer.ggml.padding_token_id
Copying tokenizer.ggml.add_bos_token
Copying tokenizer.ggml.add_eos_token
Copying tokenizer.chat_template
Copying general.quantization_version
Skipping tensor blk.0.ffn_gate.weight
Skipping tensor blk.0.ffn_up.weight
Skipping tensor blk.0.ffn_down.weight
Skipping tensor blk.1.ffn_gate.weight
Skipping tensor blk.1.ffn_up.weight
Skipping tensor 

In [22]:
!ls -ghl models/

total 23G
-rw-r--r-- 1 root 7.2G Jan  4 02:33 mistral-7b-instruct-v0.2.Q8_0.gguf
-rw-r--r-- 1 root 1.6G Jan  4 03:03 mistral-7b-instruct-v0.2.Q8_0_out.gguf
-rw-r--r-- 1 root 2.8G Jan  4 02:23 phi-2.Q8_0.gguf
-rw-r--r-- 1 root  11G Jan  4 02:43 solar-10.7b-instruct-v1.0-uncensored.Q8_0.gguf


# LLama.cpp

**作者定义的项目起源hack**: (https://github.com/ggerganov/llama.cpp/issues/33)

这是关于实现（也称为“hack”）流程的简要摘要，如果有人感兴趣的话，可能对移植其他模型会有帮助：

- 从 `ggml` 仓库的 [GPT-J](https://github.com/ggerganov/ggml/tree/master/examples/gpt-j) 示例开始
- 使用了 `ggml` 的 [4-bit 分支](https://github.com/ggerganov/ggml/pull/27)，因为它具有我们想要的初始量化支持
- LLaMA 模型与 GPT-J 有非常相似的架构。它使用相同的位置编码（RoPE），类似的激活函数（SiLU 而不是 GELU）。主要区别在于：
  - 没有偏置张量
  - 一些新的归一化层
  - 前馈部分的额外张量
  - 操作顺序稍有不同
  - 看起来上下文大小并非固定？（如果我正确理解了代码）
- 所有这些都是微不足道的更改，只需查看原始的 Python LLaMA 代码即可应用到 GPT-J 示例上
- 修改了 Python 转换脚本，以读取 7B 模型的 `.pth` 文件，并像通常一样将其转储为 `ggml` 格式
- 分词器显然更加复杂和有问题，但进行了快速的修改，至少部分支持它
- 这足以使 LLaMA-7B 正常运行。后来，通过查找到的一些来自社区的[参考资料](https://github.com/ggerganov/llama.cpp/issues/1)，使其余的模型也得到了支持

这是在 `ggml` 仓库中的 LLaMA WIP 分支，后来迁移到了 `llama.cpp`：
https://github.com/ggerganov/ggml/tree/llama

在这个过程中，甚至不需要运行原始的 Python 代码。缺点是我没有机会比较推理不同阶段的输出，所以我对这个实现的正确性有些怀疑。然而，从生成的输出来看，我猜应该是正确的。

In [23]:
%cd /content

/content


In [24]:
!git clone https://github.com/ggerganov/llama.cpp.git

Cloning into 'llama.cpp'...
remote: Enumerating objects: 15047, done.
remote: Counting objects: 100% (1943/1943), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 15047 (delta 1881), reused 1891 (delta 1858), pack-reused 13104
Receiving objects: 100% (15047/15047), 18.04 MiB | 24.83 MiB/s, done.
Resolving deltas: 100% (10435/10435), done.


## build

In [25]:
%cd llama.cpp

/content/llama.cpp


In [26]:
!make

I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi
I NVCCFLAGS:  
I LDFLAGS:    
I CC:        cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
I CXX:       g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0

cc  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prot

## BLAS 构建
使用 BLAS 支持构建程序可能会在批处理大小大于 32（默认为 512）时提高提示处理的性能。对于仅基于 CPU 的 BLAS 实现的支持不会影响正常的生成性能。对于涉及 GPU 的 BLAS 实现（例如 cuBLAS、hipBLAS 和 CLBlast），我们可能会看到生成性能的提升。目前有几种不同的 BLAS 实现可用于构建和使用：
### OpenBLAS

In [28]:
!make clean
!make LLAMA_OPENBLAS=1

I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi
I NVCCFLAGS:  
I LDFLAGS:    
I CC:        cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
I CXX:       g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0

rm -vrf *.o tests/*.o *.so *.dll benchmark-matmult common/build-info.cpp *.dot *.gcno tests/*.gcno *.gcda tests/*.gcda *.gcov tests/*.gcov lcov-report gcovr-repor

## BLIS 构建

https://github.com/ggerganov/llama.cpp/blob/master/docs/BLIS.md


In [29]:
%cd /content

/content


In [30]:
# Compile BLIS:
%%bash
git clone https://github.com/flame/blis
cd blis
./configure --enable-cblas -t openmp,pthreads auto
# will install to /usr/local/ by default.
make -j

configure: detected Linux kernel version 6.1.58+.
configure: python interpreter search list is: python python3 python2.
configure: found 'python'.
configure: using 'python' as python interpreter.
configure: found python version 3.10.12 (maj: 3, min: 10, rev: 12).
configure: python 3.10.12 appears to be supported.
configure: C compiler search list is: gcc clang cc.
configure: found 'gcc'.
configure: using 'gcc' as C compiler.
configure: found gcc version 11.4.0 (maj: 11, min: 4, rev: 0).
configure: checking for blacklisted configurations due to gcc 11.4.0.
configure: checking gcc 11.4.0 against known consequential version ranges.
configure: found assembler ('as') version 2.38 (maj: 2, min: 38, rev: ).
configure: checking for blacklisted configurations due to as 2.38.
configure: C++ compiler search list is: g++ clang++ c++.
configure: found 'g++'.
configure: using 'g++' as C++ compiler.
configure: Fortran compiler search list is: gfortran ifort.
configure: found 'gfortran'.
configure: us

Cloning into 'blis'...
In file included from ref_kernels/3/bli_gemm_ref.c:35:
ref_kernels/3/bli_gemm_ref.c: In function ‘bli_cgemm_haswell_ref’:
./frame/include//bli_complex_macro_defs.h:133:30: warning: ‘ab[<unknown>].real’ may be used uninitialized [-Wmaybe-uninitialized]
  133 | #define bli_cimag( x )  ( (x).imag )
./frame/include/level0/ri//bli_xpbyris.h:48:38: note: in definition of macro ‘bli_cxxpbyris’
   48 |         const __typeof__(yi) yt_i = (xi) + (bi) * (yr) + (br) * (yi); \
      |                                      ^~
./frame/include/level0//bli_xpbys.h:106:49: note: in expansion of macro ‘bli_creal’
  106 | #define bli_zccxpbys( x, b, y )  bli_cxxpbyris( bli_zreal(x), bli_zimag(x), bli_creal(b), bli_cimag(b), bli_creal(y), bli_cimag(y) )
      |                                                 ^~~~~~~~~
./frame/include/level0//bli_xpbys.h:187:32: note: in expansion of macro ‘bli_cccxpbys’
  187 | #define bli_zxpbys( x, b, y )  bli_zzzxpbys( x, b, y )
      |           

In [32]:
!cd blis && sudo make install

Installing libblis.a into /usr/local/lib/
Installing libblis.so.4.0.0 into /usr/local/lib/
Installing symlink libblis.so into /usr/local/lib/
Installing symlink libblis.so.4 into /usr/local/lib/
Generating monolithic cblas.h........
Generated include/haswell/cblas.h
Installing blis.h cblas.h into /usr/local/include/blis/
Installing blis.h helper header into /usr/local/include/
Installing cblas.h helper header into /usr/local/include/
Installing config.mk common.mk into /usr/local/share/blis/
Installing config/haswell/make_defs.mk into /usr/local/share/blis/config/haswell
Installing blis.pc into /usr/local/share/pkgconfig/


In [33]:
!cd llama.cpp && make clean
!cd llama.cpp && make LLAMA_BLIS=1

I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi
I NVCCFLAGS:  
I LDFLAGS:    
I CC:        cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
I CXX:       g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0

rm -vrf *.o tests/*.o *.so *.dll benchmark-matmult common/build-info.cpp *.dot *.gcno tests/*.gcno *.gcda tests/*.gcda *.gcov tests/*.gcov lcov-report gcovr-repor

## Intel oneMKL
use MKL (Math Kernel Library), a Basic Linear Algebra Subprograms (BLAS) Library
https://www.intel.com/content/www/us/en/docs/onemkl/get-started-guide/2024-0/overview.html

compiler:
https://www.intel.com/content/www/us/en/docs/dpcpp-cpp-compiler/get-started-guide/2024-0/get-started-on-linux.html



In [35]:
%cd /content

/content


In [37]:
# 安装oneApi
%%bash
wget -O- https://apt.repos.intel.com/intel-gpg-keys/GPG-PUB-KEY-INTEL-SW-PRODUCTS.PUB \
    | gpg --dearmor | sudo tee /usr/share/keyrings/oneapi-archive-keyring.gpg > /dev/null
echo "deb [signed-by=/usr/share/keyrings/oneapi-archive-keyring.gpg] https://apt.repos.intel.com/oneapi all main" | sudo tee /etc/apt/sources.list.d/oneAPI.list
apt-get update && \
    apt-get install intel-oneapi-mkl


deb [signed-by=/usr/share/keyrings/oneapi-archive-keyring.gpg] https://apt.repos.intel.com/oneapi all main
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://apt.repos.intel.com/oneapi all InRelease [4,455 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 https://apt.repos.intel.com/oneapi all/main amd64 Packages [455 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:11 https://apt.repos.intel.com/oneapi all/main all Packages [129 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy

--2024-01-04 03:45:23--  https://apt.repos.intel.com/intel-gpg-keys/GPG-PUB-KEY-INTEL-SW-PRODUCTS.PUB
Resolving apt.repos.intel.com (apt.repos.intel.com)... 23.38.16.172, 2a02:26f0:b200:38d::4b23, 2a02:26f0:b200:38e::4b23
Connecting to apt.repos.intel.com (apt.repos.intel.com)|23.38.16.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4738 (4.6K) [application/vnd.exstream-package]
Saving to: ‘STDOUT’

     0K ....                                                  100% 2.11G=0s

2024-01-04 03:45:23 (2.11 GB/s) - written to stdout [4738/4738]



In [38]:
!source /opt/intel/oneapi/mkl/latest/env/vars.sh && echo $MKLROOT

/opt/intel/oneapi/mkl/2024.0


### tips:
download offline pkg to install (apt install have not include files :{ ), have include files, use ldconfig to linker for #include<mkl.h>

GettingStarted link: https://www.intel.com/content/www/us/en/docs/onemkl/get-started-guide/current/overview.html

In [40]:
!apt autoremove intel-oneapi-mkl
#!wget https://registrationcenter-download.intel.com/akdlm/IRC_NAS/adb8a02c-4ee7-4882-97d6-a524150da358/l_onemkl_p_2023.2.0.49497_offline.sh
!wget https://registrationcenter-download.intel.com/akdlm/IRC_NAS/163da6e4-56eb-4948-aba3-debcec61c064/l_BaseKit_p_2024.0.1.46_offline.sh


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be REMOVED:
  intel-oneapi-common-licensing-2024.0 intel-oneapi-common-oneapi-vars-2024.0
  intel-oneapi-common-vars intel-oneapi-compiler-dpcpp-cpp-runtime-2024.0
  intel-oneapi-compiler-shared-runtime-2024.0 intel-oneapi-mkl intel-oneapi-mkl-2024.0
  intel-oneapi-mkl-common-2024.0 intel-oneapi-openmp-2024.0 intel-oneapi-openmp-common-2024.0
  intel-oneapi-tbb-2021.11 intel-oneapi-tbb-common-2021.11 intel-oneapi-tcm-1.0
0 upgraded, 0 newly installed, 13 to remove and 30 not upgraded.
After this operation, 2,494 MB disk space will be freed.
(Reading database ... 122084 files and directories currently installed.)
Removing intel-oneapi-mkl (2024.0.0-49656) ...
Removing intel-oneapi-mkl-2024.0 (2024.0.0-49656) ...
Removing intel-oneapi-compiler-dpcpp-cpp-runtime-2024.0 (2024.0.2-49895) ...
Removing intel-oneapi-compiler-shared-runtime-2024.0 (2024.0.2-49895) ...
Rem

In [53]:
# oneApi 组件
# https://www.intel.com/content/www/us/en/docs/oneapi/installation-guide-linux/2024-0/install-with-command-line.html
!sh l_BaseKit_p_2024.0.1.46_offline.sh -a --list-components

Extract l_BaseKit_p_2024.0.1.46_offline to /content/l_BaseKit_p_2024.0.1.46_offline...
[#################################################################################################]
Extract l_BaseKit_p_2024.0.1.46_offline completed!
Checking system requirements...
Done.
Wait while the installer is preparing...
Done.
Launching the installer...
ID                                  Version         Installed Name                                                  
intel.oneapi.lin.dpcpp-ct           2024.0.0+49391  false     Intel® DPC++ Compatibility Tool                       
intel.oneapi.lin.dpcpp_dbg          2024.0.0+49348  false     Intel® Distribution for GDB*                          
intel.oneapi.lin.dpl                2022.3.0+49384  false     Intel® oneAPI DPC++ Library                           
intel.oneapi.lin.tbb.devel          2021.11.0+49525 false     Intel® oneAPI Threading Building Blocks               
intel.oneapi.lin.ccl.devel          2021.11.2+5     false     Int

In [52]:
# 卸载oneapi
!sh l_BaseKit_p_2024.0.1.46_offline.sh -a --action remove --eula accept -s


Extract l_BaseKit_p_2024.0.1.46_offline to /content/l_BaseKit_p_2024.0.1.46_offline...
[#################################################################################################]
Extract l_BaseKit_p_2024.0.1.46_offline completed!
Checking system requirements...
Done.
Wait while the installer is preparing...
Done.
Launching the installer...
Start installation flow...
Log files: /tmp/unknown_user/intel_oneapi_installer/2024.01.04.04.35.42.059
Removal has successfully completed
Remove extracted files: /content/l_BaseKit_p_2024.0.1.46_offline...


In [54]:
# 安装 MKL
#!sh l_onemkl_p_2023.2.0.49497_offline.sh -a --components intel.oneapi.lin.mkl.devel --action install --eula accept -s
!sh l_BaseKit_p_2024.0.1.46_offline.sh -a --action install --eula accept -s \
  --components intel.oneapi.lin.mkl.devel:intel.oneapi.lin.dpcpp-cpp-compiler


Extract l_BaseKit_p_2024.0.1.46_offline to /content/l_BaseKit_p_2024.0.1.46_offline...
[#################################################################################################]
Extract l_BaseKit_p_2024.0.1.46_offline completed!
Checking system requirements...
Done.
Wait while the installer is preparing...
Done.
Launching the installer...
Start installation flow...
Installed Location: /opt/intel/oneapi
Log files: /tmp/unknown_user/intel_oneapi_installer/2024.01.04.04.42.06.327
Installation has successfully completed
Remove extracted files: /content/l_BaseKit_p_2024.0.1.46_offline...


In [55]:
!echo "/opt/intel/oneapi/mkl/latest/lib/intel64" > /etc/ld.so.conf.d/mkl.conf && ldconfig && touch /etc/profile.d/intel.sh \
    && echo ".  /opt/intel/oneapi/mkl/latest/env/vars.sh" >> /etc/profile.d/intel.sh && . /etc/profile.d/intel.sh

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /opt/intel/oneapi/mkl/latest/lib/intel64/libmkl_sycl.so is not an ELF file - it has the wrong magic bytes at the start.



In [61]:
!/opt/intel/oneapi/2024.0/bin/icx --version

Intel(R) oneAPI DPC++/C++ Compiler 2024.0.2 (2024.0.2.20231213)
Target: x86_64-unknown-linux-gnu
Thread model: posix
InstalledDir: /opt/intel/oneapi/compiler/2024.0/bin/compiler
Configuration file: /opt/intel/oneapi/compiler/2024.0/bin/compiler/../icx.cfg


In [60]:
!/opt/intel/oneapi/2024.0/bin/icpx --version

Intel(R) oneAPI DPC++/C++ Compiler 2024.0.2 (2024.0.2.20231213)
Target: x86_64-unknown-linux-gnu
Thread model: posix
InstalledDir: /opt/intel/oneapi/compiler/2024.0/bin/compiler
Configuration file: /opt/intel/oneapi/compiler/2024.0/bin/compiler/../icpx.cfg


### build llama.cpp with oneMKL

使用 oneAPI 编译器构建将使不支持 avx512 和 avx512_vnni 的英特尔处理器可以使用 avx_vnni 指令集。

请查看 [在英特尔 CPU 上优化和运行 LLaMA2](https://www.intel.com/content/www/us/en/content-details/791610/optimizing-and-running-llama2-on-intel-cpu.html) 获取更多信息。

In [62]:
!source /opt/intel/oneapi/mkl/latest/env/vars.sh && echo $MKLROOT && echo $CPATH
!cd /content/llama.cpp && rm -rf build && mkdir build && cd build \
  && cmake .. -DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=Intel10_64lp \
    -DCMAKE_C_COMPILER=/opt/intel/oneapi/2024.0/bin/icx \
    -DCMAKE_CXX_COMPILER=/opt/intel/oneapi/2024.0/bin/icpx -DLLAMA_NATIVE=ON \
  && cmake --build . --config Release

/opt/intel/oneapi/mkl/2024.0
/opt/intel/oneapi/mkl/2024.0/include:
-- The C compiler identification is IntelLLVM 2024.0.2
-- The CXX compiler identification is IntelLLVM 2024.0.2
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /opt/intel/oneapi/2024.0/bin/icx - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /opt/intel/oneapi/2024.0/bin/icpx - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1") 
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE  
-- Looking for sgemm_
-- Looking for sgemm_ - found
-- Found BLAS: /usr/lib/x86_64-linux-gnu/libmkl_intel_lp64.so;/usr/lib/x86_64-linux-gnu/libmkl_intel_thread.so;/usr/lib/x86_64-linux-gnu/libmkl_

In [66]:

!lscpu | grep avx
!lscpu | grep avx2
# no avx_vnni
!lscpu | grep avx_vnni

Flags:                              fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
Flags:                              fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed ad

## inference

In [68]:
%cd /content/

/content


In [69]:
!llama.cpp/build/bin/main -m gguf-tools/models/phi-2.Q8_0.gguf --color \
  -c 4096 --temp 0.7 --repeat_penalty 1.1 -n -1 \
  --multiline-input \
  -p "[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n{prompt}[/INST]"



Log start
main: build = 1761 (cb1e281)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1704344933
llama_model_loader: loaded meta data with 20 key-value pairs and 325 tensors from gguf-tools/models/phi-2.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi2
llama_model_loader: - kv   1:                               general.name str              = Phi2
llama_model_loader: - kv   2:                        phi2.context_length u32              = 2048
llama_model_loader: - kv   3:                      phi2.embedding_length u32              = 2560
llama_model_loader: - kv   4:                   phi2.feed_forward_length u32              = 10240
llama_model_loader: - kv   5:                           phi2.block_count u32              = 32
llama_model_loader: - kv   6:      

In [73]:
!llama.cpp/build/bin/main -m gguf-tools/models/llama-2-7b-chat.Q4_K_M.gguf --color \
  -c 4096 --temp 0.7 --repeat_penalty 1.1 -n -1 \
  --multiline-input \
  -i -ins


Log start
main: build = 1761 (cb1e281)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1704345853
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from gguf-tools/models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   